In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np

In [2]:
with open(r'api_key.txt', 'r') as file:
    api_key = file.readline()

In [3]:
url_austin = r'https://en.wikipedia.org/wiki/List_of_Austin_neighborhoods'

In [4]:
table_austin = pd.read_html(url_austin)
table_austin = table_austin[0]
table_austin.drop(['COA ID#[nb 1]'], axis=1, inplace = True)
austin_neighborhoods = table_austin
austin_neighborhoods.rename(columns={'Name':'Neighborhood'}, inplace=True)
austin_neighborhoods.head()

,Neighborhood
0,Bryker Woods
1,Caswell Heights
2,Downtown Austin
3,Eastwoods
4,Hancock


In [47]:
url_dallas = r'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Dallas'

In [81]:
table_dallas = pd.read_html(url_dallas)
table_dallas_concat = pd.DataFrame(columns=["Neighborhood"])
for i in range(1, len(table_dallas)):
    temp_table = pd.DataFrame(table_dallas[i].iloc[:,1])
    temp_table.rename(columns={temp_table.columns[0]:"Neighborhood"}, inplace=True)
    table_dallas_concat = table_dallas_concat.append(temp_table, ignore_index=True)
    temp_table = []
dallas_neighborhoods = table_dallas_concat
dallas_neighborhoods.head()

,Neighborhood
0,Baylor District
1,The Cedars
2,Civic Center District
3,Dallas Arts District
4,Dallas Farmers Market


In [72]:
def remove_website_str(x):
    if x.startswith('website'):
        x = x.split()
        x=' '.join(x[1:])
    return x

In [82]:
dallas_neighborhood_clean = dallas_neighborhoods['Neighborhood'].apply(remove_website_str)
dallas_neighborhood_clean = pd.DataFrame(dallas_neighborhood_clean, columns = ['Neighborhood'])
dallas_neighborhood_clean.drop(index=[205, 206, 207, 208, 209], inplace = True)
dallas_neighborhood_clean.head()

,Neighborhood
0,Baylor District
1,The Cedars
2,Civic Center District
3,Dallas Arts District
4,Dallas Farmers Market


In [84]:
dallas_neighborhood_clean['Neighborhood'] = dallas_neighborhood_clean['Neighborhood'].map(lambda x: x +', Dallas, '+ 'TX')
dallas_neighborhood_clean.head()

,Neighborhood
0,"Baylor District, Dallas, TX"
1,"The Cedars, Dallas, TX"
2,"Civic Center District, Dallas, TX"
3,"Dallas Arts District, Dallas, TX"
4,"Dallas Farmers Market, Dallas, TX"


In [7]:
url_chicago = r'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'
#url_chicago = r'https://en.wikipedia.org/w/index.php?title=List_of_neighborhoods_in_Chicago&action=edit&section=1'

In [8]:
table_chicago = pd.read_html(url_chicago)
table_chicago = table_chicago[0]
table_chicago.drop(['Community area'], axis=1, inplace=True)
chicago_neighborhoods = table_chicago
chicago_neighborhoods.head()

,Neighborhood
0,Albany Park
1,Altgeld Gardens
2,Andersonville
3,Archer Heights
4,Armour Square


In [23]:
chicago_neighborhoods['Neighborhood'] = chicago_neighborhoods['Neighborhood'].map(lambda x: x +', Chicago, '+ 'IL')
chicago_neighborhoods.head()

,Neighborhood
0,"Albany Park, Chicago, IL"
1,"Altgeld Gardens, Chicago, IL"
2,"Andersonville, Chicago, IL"
3,"Archer Heights, Chicago, IL"
4,"Armour Square, Chicago, IL"


In [9]:
import re
import requests
from bs4 import BeautifulSoup

In [26]:
jacksonville_neighborhoods = pd.DataFrame(columns=['Neighborhood'])
jacksonville_neighborhoods

,Neighborhood


In [27]:
url_string=r'https://www.conservapedia.com/Neighborhoods_of_Jacksonville,_Florida'
page=requests.get(url_string)
page=page.text
soup=BeautifulSoup(page,features="html.parser")
results=soup.find_all("li", class_="toclevel-2")
for index, item in enumerate(results):
    area = item.find("span", class_="toctext").text
    jacksonville_neighborhoods = jacksonville_neighborhoods.append({'Neighborhood':area}, ignore_index=True) 
jacksonville_neighborhoods.head()
#for item in results:
#    print(item.text)

,Neighborhood
0,Argyle
1,Avondale
2,Cedar Hills
3,Confederate Point
4,Lake Shore


In [28]:
jacksonville_neighborhoods['Neighborhood'] = jacksonville_neighborhoods['Neighborhood'].map(lambda x: x +', Jacksonville, '+ 'FL')
#mod_jacksonville_neighborhoods.rename(columns={0:'Neighborhood'}, inplace=True)
jacksonville_neighborhoods.head()

,Neighborhood
0,"Argyle, Jacksonville, FL"
1,"Avondale, Jacksonville, FL"
2,"Cedar Hills, Jacksonville, FL"
3,"Confederate Point, Jacksonville, FL"
4,"Lake Shore, Jacksonville, FL"


In [13]:
import json
from pandas.io.json import json_normalize

In [29]:
def get_lng_lat_zip(neighborhoods):
    location_lng_lat=pd.DataFrame(columns=['Latitude', 'Longitude'])
    for i in range(len(neighborhoods)):
        url = r'https://maps.googleapis.com/maps/api/geocode/json?address={}&key={}'.format(neighborhoods['Neighborhood'][i], api_key)
        r = requests.get(url)
        json_r = json.loads(r.text)
        try:
            location_pd = json_normalize(json_r['results'])
            location_lng_lat_temp =  location_pd[['geometry.location.lat', 'geometry.location.lng']]
            location_lng_lat_temp.columns=['Latitude', 'Longitude']
            location_lng_lat = pd.concat([location_lng_lat, location_lng_lat_temp], ignore_index=True)
        except:
            continue
            
    return location_lng_lat                              

In [30]:
jacksonville_neighborhoods_zip = get_lng_lat_zip(jacksonville_neighborhoods)
jacksonville_neighborhoods_zip.head()

,Latitude,Longitude
0,30.196921,-81.756282
1,30.296769,-81.710315
2,30.253614,-81.756138
3,30.256893,-81.736310
4,30.280630,-81.726734


In [31]:
jacksonville_nb_lng_lat = pd.concat([jacksonville_neighborhoods,
                                     jacksonville_neighborhoods_zip],
                                    axis=1)
jacksonville_nb_lng_lat.head()

,Neighborhood,Latitude,Longitude
0,"Argyle, Jacksonville, FL",30.196921,-81.756282
1,"Avondale, Jacksonville, FL",30.296769,-81.710315
2,"Cedar Hills, Jacksonville, FL",30.253614,-81.756138
3,"Confederate Point, Jacksonville, FL",30.256893,-81.736310
4,"Lake Shore, Jacksonville, FL",30.280630,-81.726734


In [32]:
def zip_nb(neighborhoods):
    zip_pd_temp = []
    for i in range(len(neighborhoods)):
        url_2 = 'https://maps.googleapis.com/maps/api/geocode/json?latlng={},{}&key={}'.format(neighborhoods['Latitude'][i], neighborhoods['Longitude'][i],api_key)
        r = requests.get(url_2)
        json_r = json.loads(r.text)
        location_pd_3 = json_normalize(json_r['results'][0]['address_components'])
        count=0
        for line in location_pd_3.values:
            if 'postal_code' in line[2]:
                zip_code = line[0]
                count=1
            if count != 1:
                zip_code = 'NA'    
        zip_pd_temp.append(zip_code)
        count=0
    zip_pd = pd.DataFrame(zip_pd_temp, columns = ['zip_code'])
    return(zip_pd)  

In [33]:
jacksonville_zip_pd = zip_nb(jacksonville_nb_lng_lat)
jacksonville_zip_pd.head()

,zip_code
0,32244
1,32205
2,32210
3,32210
4,32210


In [34]:
jacksonville_zip_lng_lat = pd.concat([jacksonville_nb_lng_lat,
                                     jacksonville_zip_pd],
                                    axis=1)
jacksonville_zip_lng_lat.head()

,Neighborhood,Latitude,Longitude,zip_code
0,"Argyle, Jacksonville, FL",30.196921,-81.756282,32244
1,"Avondale, Jacksonville, FL",30.296769,-81.710315,32205
2,"Cedar Hills, Jacksonville, FL",30.253614,-81.756138,32210
3,"Confederate Point, Jacksonville, FL",30.256893,-81.736310,32210
4,"Lake Shore, Jacksonville, FL",30.280630,-81.726734,32210


In [35]:
jacksonville_zip_lng_lat.to_csv('jacksonville_zip_lng_lat.csv', index='False')

In [86]:
dallas_neighborhoods_zip = get_lng_lat_zip(dallas_neighborhood_clean)
dallas_neighborhoods_zip.head()

,Latitude,Longitude
0,32.787883,-96.798620
1,32.765124,-96.789371
2,32.774389,-96.800285
3,32.789731,-96.798262
4,32.777495,-96.789263


In [202]:
dallas_nb_lng_lat = pd.concat([dallas_neighborhood_clean,
                                     dallas_neighborhoods_zip],
                                    axis=1)
dallas_nb_lng_lat.head()

,Neighborhood,Latitude,Longitude
0,"Baylor District, Dallas, TX",32.787883,-96.798620
1,"The Cedars, Dallas, TX",32.765124,-96.789371
2,"Civic Center District, Dallas, TX",32.774389,-96.800285
3,"Dallas Arts District, Dallas, TX",32.789731,-96.798262
4,"Dallas Farmers Market, Dallas, TX",32.777495,-96.789263


In [90]:
dallas_zip_pd = zip_nb(dallas_nb_lng_lat)
dallas_zip_pd.head()

,zip_code
0,75201
1,75215
2,75202
3,75201
4,75201


In [205]:
dallas_zip_lng_lat = pd.concat([dallas_nb_lng_lat,
                                     dallas_zip_pd],
                                    axis=1)
print(dallas_zip_lng_lat.shape)
dallas_zip_lng_lat.head().head()

(221, 4)


,Neighborhood,Latitude,Longitude,zip_code
0,"Baylor District, Dallas, TX",32.787883,-96.798620,75201
1,"The Cedars, Dallas, TX",32.765124,-96.789371,75215
2,"Civic Center District, Dallas, TX",32.774389,-96.800285,75202
3,"Dallas Arts District, Dallas, TX",32.789731,-96.798262,75201
4,"Dallas Farmers Market, Dallas, TX",32.777495,-96.789263,75201


In [109]:
index_list = []
for i in range(206,221):
    index_list.append(int(i))

In [111]:
dallas_zip_lng_lat.drop(index=index_list, inplace=True)
dallas_zip_lng_lat.head()

,Neighborhood,Latitude,Longitude,zip_code
0,"Baylor District, Dallas, TX",32.787883,-96.798620,75201
1,"The Cedars, Dallas, TX",32.765124,-96.789371,75215
2,"Civic Center District, Dallas, TX",32.774389,-96.800285,75202
3,"Dallas Arts District, Dallas, TX",32.789731,-96.798262,75201
4,"Dallas Farmers Market, Dallas, TX",32.777495,-96.789263,75201


In [ ]:
dallas_zip_lng_lat.to_csv('dallas_zip_lng_lat.csv', index='False')

In [340]:
def zip_demographics(neighborhood_zip):
    zip_list = []
    zip_list_nan = ['N/A' for i in range(37)]
    for i in range(len(neighborhood_zip)):
        url_zip = 'https://zipwho.com/?zip={}-&city=&filters=--_--_--_--&state=&mode=zip'.format(neighborhood_zip['zip_code'][i])
        headers ={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0'}
        page=requests.get(url_zip, headers=headers)
        page=page.text
        soup=BeautifulSoup(page,features="html.parser")
        start_index = str(soup.contents[4]).index('return')
        end_index = str(soup.contents[4]).index('}')
        try:
            string_data = str(soup.contents[4])[start_index+8:end_index-4]
            string_data_split = string_data.split('\\n')
            data = string_data_split[1].split(',')
            numbers = string_data_split[0].split(',')
            zip_list.append(data)
        except:
            zip_list.append(zip_list_nan)
    zip_list_pd = pd.DataFrame(zip_list, columns = numbers)
    zip_list_pd.rename(columns ={'zip':'zip_code'}, inplace = True)
    zip_list_pd.drop(columns=['city', 'state'], inplace=True)
    return zip_list_pd

In [341]:
jacksonville_zip_list = zip_demographics(jacksonville_zip_lng_lat)
jacksonville_zip_list.head()

,zip_code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,32244,41769,59,89.6,40,21.0,49,70.9,31,5.4,43,14.0,42,25.8,40,46852,96,2.7,74,31.8,11,91.3,45,58.0,33,15.0,97,65.7,21,20.8,85,4.6,90,5.4,72
1,32205,34579,35,76.4,28,20.0,34,58.0,13,5.3,33,21.7,67,33.6,70,30128,86,2.2,8,35.2,30,88.8,31,47.5,9,15.7,98,71.0,25,20.1,85,2.8,84,3.2,61
2,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70
3,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70
4,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70


In [276]:
jacksonville_zip_lng_lat['zip_code'] = pd.to_numeric(jacksonville_zip_lng_lat['zip_code'])
jacksonville_zip_list['zip_code'] = pd.to_numeric(jacksonville_zip_list['zip_code'])

In [343]:
jacksonville_data_merged = pd.merge(jacksonville_zip_lng_lat, jacksonville_zip_list, right_index=True, left_index=True)
jacksonville_data_merged = jacksonville_data_merged.drop('zip_code_x', axis=1)
jacksonville_data_merged.rename(columns={'zip_code_y':'Zip Code'}, inplace=True)
jacksonville_data_merged.head()

,Neighborhood,Latitude,Longitude,Zip Code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,"Argyle, Jacksonville, FL",30.196921,-81.756282,32244,41769,59,89.6,40,21.0,49,70.9,31,5.4,43,14.0,42,25.8,40,46852,96,2.7,74,31.8,11,91.3,45,58.0,33,15.0,97,65.7,21,20.8,85,4.6,90,5.4,72
1,"Avondale, Jacksonville, FL",30.296769,-81.710315,32205,34579,35,76.4,28,20.0,34,58.0,13,5.3,33,21.7,67,33.6,70,30128,86,2.2,8,35.2,30,88.8,31,47.5,9,15.7,98,71.0,25,20.1,85,2.8,84,3.2,61
2,"Cedar Hills, Jacksonville, FL",30.253614,-81.756138,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70
3,"Confederate Point, Jacksonville, FL",30.256893,-81.736310,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70
4,"Lake Shore, Jacksonville, FL",30.280630,-81.726734,32210,39095,51,79.4,30,20.0,34,63.2,18,5.5,49,17.9,57,26.4,43,58345,98,2.5,46,33.8,20,88.9,32,54.3,21,14.1,94,65.4,21,24.3,87,2.8,84,4.9,70


In [344]:
jacksonville_filtered = jacksonville_data_merged.sort_values(by=['MedianIncome'],ascending=False)
jacksonville_filtered.head()

,Neighborhood,Latitude,Longitude,Zip Code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
18,"Mandarin, Jacksonville, FL",30.152750,-81.652822,32223,66079,91,146.9,73,19.6,27,82.7,72,6.4,90,37.9,87,45.3,89,25424,81,2.6,59,38.1,61,91.4,45,66.4,81,10.7,69,88.7,49,4.5,65,1.6,75,3.5,63
16,"Baymeadows, Jacksonville, FL",30.236722,-81.559158,32256,45460,67,198.0,86,20.1,36,31.9,2,4.3,4,45.2,92,49.5,92,29254,85,2.0,2,31.4,10,96.8,72,49.2,11,13.0,90,71.5,25,12.0,79,7.0,94,6.5,75
19,"Sandalwood, Jacksonville, FL",30.310517,-81.508249,32246,43672,63,90.8,41,20.9,47,69.7,28,5.5,50,17.1,54,26.8,45,36012,91,2.7,73,31.7,11,93.0,54,58.2,34,14.0,94,68.3,23,16.8,83,5.5,92,6.3,74
0,"Argyle, Jacksonville, FL",30.196921,-81.756282,32244,41769,59,89.6,40,21.0,49,70.9,31,5.4,43,14.0,42,25.8,40,46852,96,2.7,74,31.8,11,91.3,45,58.0,33,15.0,97,65.7,21,20.8,85,4.6,90,5.4,72
5,"Marietta, Jacksonville, FL",30.334397,-81.802480,32220,41622,58,82.8,33,20.3,38,85.8,84,5.7,66,6.0,4,18.7,9,10583,57,2.7,69,35.4,31,99.6,82,61.7,52,12.3,86,91.8,56,3.6,62,0.6,56,1.6,45


In [345]:
jacksonville_filtered.to_csv('jacksonville_filtered.csv')

In [324]:
dallas_zip_list = zip_demographics(dallas_zip_lng_lat)
dallas_zip_list.head()

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

,zip_code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
1,75215,16043,0,39.4,1,25.9,93,33.6,3,4.3,4,6.4,5,17.7,6,18643,72,2.6,58,34.4,23,78.9,4,34.3,1,13.7,93,2.4,0,84.0,99,0.1,30,12.1,84
2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
4,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77


In [328]:
dallas_data_merged = pd.merge(dallas_zip_lng_lat, dallas_zip_list, right_index=True, left_index=True)
dallas_data_merged = dallas_data_merged.drop('zip_code_y', axis=1)
dallas_data_merged.rename(columns={'zip_code_x':'Zip Code'}, inplace=True)
dallas_data_merged.head()

,Neighborhood,Latitude,Longitude,Zip Code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,"Baylor District, Dallas, TX",32.787883,-96.798620,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
1,"The Cedars, Dallas, TX",32.765124,-96.789371,75215,16043,0,39.4,1,25.9,93,33.6,3,4.3,4,6.4,5,17.7,6,18643,72,2.6,58,34.4,23,78.9,4,34.3,1,13.7,93,2.4,0,84.0,99,0.1,30,12.1,84
2,"Civic Center District, Dallas, TX",32.774389,-96.800285,75202,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,"Dallas Arts District, Dallas, TX",32.789731,-96.798262,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
4,"Dallas Farmers Market, Dallas, TX",32.777495,-96.789263,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77


In [335]:
dallas_filter = dallas_data_merged['MedianIncome'].apply(lambda x: x != 'N/A')
dallas_filtered=dallas_data_merged[dallas_filter]
dallas_filtered.head()

,Neighborhood,Latitude,Longitude,Zip Code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
0,"Baylor District, Dallas, TX",32.787883,-96.798620,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
1,"The Cedars, Dallas, TX",32.765124,-96.789371,75215,16043,0,39.4,1,25.9,93,33.6,3,4.3,4,6.4,5,17.7,6,18643,72,2.6,58,34.4,23,78.9,4,34.3,1,13.7,93,2.4,0,84.0,99,0.1,30,12.1,84
3,"Dallas Arts District, Dallas, TX",32.789731,-96.798262,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
4,"Dallas Farmers Market, Dallas, TX",32.777495,-96.789263,75201,56675,84,311.3,95,15.6,0,10.8,0,3.3,1,63.5,98,66.1,99,3324,27,1.6,0,30.0,7,112.0,96,31.5,1,11.6,79,68.8,23,19.4,85,2.2,80,7.6,77
5,"Deep Ellum, Dallas, TX",32.784392,-96.780185,75226,38911,50,441.5,98,32.5,99,1.0,0,2.6,0,34.5,84,40.9,84,2693,20,2.5,33,29.6,6,184.3,99,45.2,7,9.0,43,37.8,8,11.4,78,1.8,77,48.5,96


In [338]:
dallas_filtered.sort_values(by=['MedianIncome'], inplace=True, ascending=False)
dallas_filtered.head()

C:\Users\gtm12\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Neighborhood,Latitude,Longitude,Zip Code,MedianIncome,MedianIncomeRank,CostOfLivingIndex,CostOfLivingRank,MedianMortgageToIncomeRatio,MedianMortgageToIncomeRank,OwnerOccupiedHomesPercent,OwnerOccupiedHomesRank,MedianRoomsInHome,MedianRoomsInHomeRank,CollegeDegreePercent,CollegeDegreeRank,ProfessionalPercent,ProfessionalRank,Population,PopulationRank,AverageHouseholdSize,AverageHouseholdSizeRank,MedianAge,MedianAgeRank,MaleToFemaleRatio,MaleToFemaleRank,MarriedPercent,MarriedRank,DivorcedPercent,DivorcedRank,WhitePercent,WhiteRank,BlackPercent,BlackRank,AsianPercent,AsianRank,HispanicEthnicityPercent,HispanicEthnicityRank
113,"Northwood Hills, Dallas, TX",32.863998,-96.803733,75225,98838,98,485.0,98,21.0,49,76.9,49,6.8,94,74.9,99,66.0,99,20157,74,2.3,14,41.7,88,78.4,3,65.6,77,7.9,28,95.9,72,0.0,5,0.7,58,2.2,53
12,"Uptown, Dallas, TX",32.868538,-96.773511,75225,98838,98,485.0,98,21.0,49,76.9,49,6.8,94,74.9,99,66.0,99,20157,74,2.3,14,41.7,88,78.4,3,65.6,77,7.9,28,95.9,72,0.0,5,0.7,58,2.2,53
88,"Stultz Road, Dallas, TX",32.869628,-96.793932,75225,98838,98,485.0,98,21.0,49,76.9,49,6.8,94,74.9,99,66.0,99,20157,74,2.3,14,41.7,88,78.4,3,65.6,77,7.9,28,95.9,72,0.0,5,0.7,58,2.2,53
92,"Whispering Hills, Dallas, TX",32.989294,-97.018090,75028,91855,98,179.9,83,20.3,39,92.1,97,7.3,97,51.0,94,54.5,95,36393,91,3.1,93,32.9,16,96.1,69,75.2,99,6.5,13,86.4,44,3.1,60,2.6,82,5.4,72
29,"Greenland Hills, Dallas, TX",33.051367,-97.070180,75028,91855,98,179.9,83,20.3,39,92.1,97,7.3,97,51.0,94,54.5,95,36393,91,3.1,93,32.9,16,96.1,69,75.2,99,6.5,13,86.4,44,3.1,60,2.6,82,5.4,72


In [346]:
dallas_filtered.to_csv('dallas_filtered.to_csv')